<a href="https://colab.research.google.com/github/dlsnfl03/Llama_fine-tuning/blob/main/llm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers accelerate datasets peft trl bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import sys
# import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    HfArgumentParser,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TextStreamer,
    logging as hf_logging,
)
import logging
from trl import SFTTrainer, SFTConfig

from trl.trainer import ConstantLengthDataset

# Setting

In [ ]:
base_model_id = "meta-llama/Llama-3.2-1B-Instruct"
device_map="cuda"
torch_dtype = torch.bfloat16
output_dir = "./meta-order-analysis"
dataset_name = "./llm-modeling-lab.jsonl"
seq_length = 512

# Raw Dataset

In [ ]:
from datasets import Dataset

# 전체 데이터셋 로드
full_dataset = Dataset.from_json('/content/drive/MyDrive/Colab Notebooks/자연어처리/llm-modeling-lab.jsonl')

# 2800개는 학습, 200개는 검증 세트로 나누기
dataset_split = full_dataset.train_test_split(train_size=2800, test_size=200, seed=42)
train_data = dataset_split['train']
validation_data = dataset_split['test']


Generating train split: 0 examples [00:00, ? examples/s]

# Loading the Tokenizer

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi, login

# Hugging Face API 토큰 불러오기
hf_api_token = userdata.get('HUGGINGFACE_API_TOKEN')
if hf_api_token:
    # Hugging Face 로그인
    login(token=hf_api_token)
    print("Successfully logged into Hugging Face.")
else:
    print("HUGGINGFACE_API_TOKEN not found in Colab secrets")


Successfully logged into Hugging Face.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id
)
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Loading the Base Model

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
base_model.config.use_cache = False

In [ ]:
if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if base_model.config.pad_token_id != tokenizer.pad_token_id:
    base_model.config.pad_token_id = tokenizer.pad_token_id

# Utilities

In [ ]:
def chars_token_ratio(dataset, tokenizer, prepare_sample_text, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [ ]:
def function_prepare_sample_text(tokenizer, for_train=True):
    """A Closure"""

    def _prepare_sample_text(example):
        """Prepare the text from a sample of the dataset."""
        user_prompt="너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: "
        messages = [
            # {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": f"{user_prompt}{example['input']}"},
        ]
        if for_train:
            messages.append({"role": "assistant", "content": f"{example['output']}"})

        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False if for_train else True)
        return text
    return _prepare_sample_text

In [ ]:
def create_datasets(tokenizer, dataset, seq_length):

    prepare_sample_text = function_prepare_sample_text(tokenizer)

    chars_per_token = chars_token_ratio(dataset, tokenizer, prepare_sample_text)
    print(
        f"The character to token ratio of the dataset is: {chars_per_token:.2f}"
    )

    cl_dataset = ConstantLengthDataset(
        tokenizer,
        dataset,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )

    return cl_dataset

# Build a dataset

In [ ]:
ds = create_datasets(tokenizer, train_data, seq_length)
dsv = create_datasets(tokenizer, validation_data, seq_length)

100%|██████████| 400/400 [00:00<00:00, 817.40it/s]


The character to token ratio of the dataset is: 2.62


 50%|█████     | 200/400 [00:00<00:00, 1020.45it/s]

The character to token ratio of the dataset is: 2.62


In [ ]:
it = iter(ds)

In [ ]:
tokenizer.decode(next(it)['input_ids'])

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


' 결과 1: 음식명:국내산오븐구이닭다리,수량:2개<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Dec 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 제주오겹살 200g 와 복분자차 한잔주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- 분석 결과 0: 음식명:제주오겹살,옵션:200g\n- 분석 결과 1: 음식명:복분자차, 수량:한잔<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Dec 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 오늘은 국물이 땡기네요. 황태술국 한그릇 주실 수 있나요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- 분석 결과 0: 음식명:황태술국, 수량:한그릇<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nC

# Fine Tuning

## Seting for a LoRA adapter

In [ ]:
lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "down_proj",
                "up_proj",
                "gate_proj",
            ],
            bias="none",
            task_type="CAUSAL_LM",
        )

In [ ]:
peft_config = lora_config

In [ ]:
from google.colab import userdata
import wandb

wandb_api_key = userdata.get('WANDB_API_KEY')
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to Weights & Biases")
else:
    print("WANDB_API_KEY not found in Colab secrets")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to Weights & Biases


In [ ]:
sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    max_grad_norm=0.3,
    weight_decay=0.05,
    num_train_epochs=1,
    logging_steps=20,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    max_seq_length=seq_length,
    report_to="wandb",
    run_name="Llama-3.2-1B-Instruct-fine-tuning"
)

In [ ]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=ds,
    eval_dataset=None,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=sft_config
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(output_dir="/content/drive/MyDrive/Colab Notebooks/자연어처리/lora2")

In [ ]:
from huggingface_hub import HfApi

# 저장된 LoRA Adapter 경로
model_path = "/content/drive/MyDrive/Colab Notebooks/자연어처리/lora2"
# 업로드할 리포지토리 이름 (username/repo_name 형태로 작성)
repo_name = "Nuri03/20220906_lora_adapter2"

# Hugging Face Hub에 업로드
api = HfApi()
api.upload_folder(folder_path=model_path, repo_id=repo_name, repo_type="model")


# Evaluation

## Utilities for Evaluation

In [ ]:
def wrapper_generate(tokenizer, model, input_prompt, do_stream=False):
    def get_text_after_prompt(text):
        pattern = r'<start_of_turn>model\n(.*?)<end_of_turn>'
        match = re.search(pattern, text, re.DOTALL)

        if match:
            extracted_text = match.group(1).strip()
            return extracted_text
        else:
            return "매칭되는 텍스트가 없습니다."

    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer if do_stream else None,
            use_cache=True,
            max_new_tokens=128, #float("inf"),
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=True)

    # 결과에 대해 특별 처리
    return get_text_after_prompt(decoded_text[0])

## Load the Fine-Tuned Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 ID를 Hugging Face Hub에서 불러오기
adapter_model_id = "Nuri03/20220906_lora_adapter2"  # 업로드한 모델 ID로 변경
model = AutoModelForCausalLM.from_pretrained(adapter_model_id)
tokenizer = AutoTokenizer.from_pretrained(adapter_model_id)


adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="Nuri03/20220906_lora_adapter2",
    torch_dtype=torch.float32,  # 필요에 따라 변경
    device_map="auto",         # 디바이스 매핑
    cache_dir="/content/cache"  # 캐시 디렉토리 (필요시)
)


adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

In [ ]:
device_map = "cpu"
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    "Nuri03/20220906_lora_adapter2",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch_dtype,
    device_map=device_map,
    cache_dir="/content/cache"
)

In [ ]:
merged_model = trained_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained(
    '/content/drive/MyDrive/Colab Notebooks/자연어처리',
    max_shard_size="4GB",
    safe_serialization=True
)

## Release GPU memory

In [ ]:
import gc
del trainer
del base_model
gc.collect()
torch.cuda.empty_cache()

If you want other checkpoints, modify suffix.

In [ ]:
output_dir="/content/drive/MyDrive/Colab Notebooks/자연어처리/lora2"

In [ ]:
merged_model = (
    AutoPeftModelForCausalLM.from_pretrained(
        f"{output_dir}",
        quantization_config=bnb_config,
        device_map="auto",
        # device='cuda',
        trust_remote_code=True,
    )
)

## Test

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
import sacrebleu

references = []  # 실제 정답 텍스트 리스트
predictions = []  # 모델 예측 텍스트 리스트

# 검증 데이터셋에서 예측값 생성 및 BLEU 점수 계산
for example in validation_data:
    input_text = example['input']  # 입력 텍스트
    reference_text = example['output']  # 참조 텍스트

    # 모델 예측 생성
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")  # 입력 데이터를 GPU로 이동
    outputs = merged_model.generate(**inputs, max_new_tokens=50)  # 필요 시 max_new_tokens 조정
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 결과 저장
    references.append(reference_text)
    predictions.append(generated_text)

# BLEU 점수 계산
bleu = sacrebleu.corpus_bleu(predictions, [references])
print(f"BLEU Score: {bleu.score}")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

BLEU Score: 13.774604410138974


In [ ]:
preprocessor = function_prepare_sample_text(tokenizer, for_train=False)

In [ ]:
preprocessor({'input':'아이스아메리카노 그랑데 한잔 주세요'})

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Dec 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 아이스아메리카노 그랑데 한잔 주세요<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'